In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import os
simp_path = '.env'
abs_path = os.path.abspath(simp_path)
CREDENTIAL={'EMAIL':'','PASSWORD':''}
with open(abs_path,"r") as archivo:
    for linea in archivo:        
        if (str(linea.strip('\n')).split('='))[0] == "EMAIL":
            CREDENTIAL["EMAIL"]=(str(linea.strip('\n')).split('='))[1]
        if (str(linea.strip('\n')).split('='))[0] == "PASSWORD":
            CREDENTIAL["PASSWORD"]=(str(linea.strip('\n')).split('='))[1]
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
#abrir formulario


driver.get('https://sivireno.undc.edu.pe/index_login.php')
driver.find_element(by=By.XPATH, value='/html/body/div[1]/div/div/form/div[1]/a').click()
driver.find_element(by=By.XPATH, value='//*[@id="identifierId"]').send_keys(CREDENTIAL["EMAIL"])
driver.find_element(by=By.XPATH, value='//*[@id="identifierNext"]/div/button/span').click()
time.sleep(2)
driver.find_element(by=By.XPATH, value='//*[@id="password"]/div[1]/div/div[1]/input').send_keys(CREDENTIAL["PASSWORD"])
driver.find_element(by=By.XPATH, value='//*[@id="passwordNext"]/div/button/span').click()
time.sleep(2)

[WDM] - Downloading: 100%|██████████| 6.80M/6.80M [00:01<00:00, 3.87MB/s]


In [65]:
url="https://sivireno.undc.edu.pe/index_home.php?s=asistencia_cursos.php&id_cl=2771"
driver.get(url)

In [3]:
#SCRIPT CARGA DET LECTIVA DE MANERA MASIVA
import pandas as pd
import datetime
df = pd.read_excel('EXCEL_TRABAJADO/2023-1cl.xlsx')
df
semanas_xpath= {'1':'//*[@id="exampleFormControlSelect1"]/option[1]',
               '2':'//*[@id="exampleFormControlSelect1"]/option[2]',
               '3':'//*[@id="exampleFormControlSelect1"]/option[3]',
               '4':'//*[@id="exampleFormControlSelect1"]/option[4]',
               '5':'//*[@id="exampleFormControlSelect1"]/option[5]',
               '6':'//*[@id="exampleFormControlSelect1"]/option[6]',
               '7':'//*[@id="exampleFormControlSelect1"]/option[7]',
               '8':'//*[@id="exampleFormControlSelect1"]/option[8]',
               '9':'//*[@id="exampleFormControlSelect1"]/option[9]',
               '10':'//*[@id="exampleFormControlSelect1"]/option[10]',
               '11':'//*[@id="exampleFormControlSelect1"]/option[11]',
               '12':'//*[@id="exampleFormControlSelect1"]/option[12]',
               '13':'//*[@id="exampleFormControlSelect1"]/option[13]',
               '14':'//*[@id="exampleFormControlSelect1"]/option[14]',
               '15':'//*[@id="exampleFormControlSelect1"]/option[15]',
               '16':'//*[@id="exampleFormControlSelect1"]/option[16]',}

#INICIO DE CLASES
w_iclass={'LUNES':'27/03/2023',
               'MARTES':'28/03/2023',
               'MIERCOLES':'29/03/2023',
               'JUEVES':'30/03/2023',
               'VIERNES':'31/03/2023',
               'SABADO':'01/04/2023',
               'DOMINGO':'02/04/2023'}

urltemp=""
############################################            METODOS
    #ALGORITMO PARA LA FECHA
def convertDateWrite(fecha):
    f = str(fecha).split('-')
    año = f[0]
    mes = f[1]
    d= f[2].split()
    dia = d[0]
    return dia+mes+año
    
    #ALGORTIMO HORA
def convertHourWrite(hour):
    h = str(hour).split(':')
    return h[0]+h[1]

    # 7 + W, CON RESPECTO AL DIA
def sumWeek(dia,sem):
    return convertDateWrite(datetime.datetime.strptime(w_iclass[dia], '%d/%m/%Y')+(datetime.timedelta(days=7)*(sem-1)))

def writeForm(semi,semf,dia,doc,horae,horas,asig,turno,ciclo):
    x=1
    for j in range(int(semi),int(semf+1)):
        print(str(x)+") Asig: "+asig+" "+ciclo+" "+turno+" Docente :"+doc+" SEM:"+str(j)+" semi:"+str(semi)+" semf:"+str(semf)+" FECHA:"+sumWeek(dia,j))
        time.sleep(0.140)
        #ingresar semana
        driver.find_element(by=By.XPATH, value=semanas_xpath[str(int(j))]).click()
        time.sleep(0.150)
        #ingresar fecha
        driver.find_element(by=By.XPATH, value='//*[@id="asistencia_docentes"]/div[4]/div[2]/table/tbody/tr[2]/td[2]/input').send_keys(sumWeek(dia,j))

        #ingresar fecha entrada
        driver.find_element(by=By.XPATH, value='//*[@id="asistencia_docentes"]/div[4]/div[2]/table/tbody/tr[2]/td[3]/input').send_keys(str(convertHourWrite(horae)))
        #print(str(convertHourWrite(horae)))
        #ingresar fecha salida
        driver.find_element(by=By.XPATH, value='//*[@id="asistencia_docentes"]/div[4]/div[2]/table/tbody/tr[2]/td[4]/input').send_keys(str(convertHourWrite(horas)))

        #enviar registro
        driver.find_element(by=By.XPATH, value='//*[@id="asistencia_docentes"]/div[4]/div[2]/table/tbody/tr[2]/td[9]/button').click()
        time.sleep(0.100)
        x=x+1
        
############################################            METODOS/*

for row, datos in df.iterrows():
    if datos['FLAG'] == 'SI':
        url = datos['URL']
        semi = datos['SEM_I']
        semf = datos['SEM_F']
        dia = datos['DIA']
        doc=datos['DOCENTE']
        horae = datos['HORAE']
        horas = datos['HORAS']
        asig=datos['ASIGNATURA']
        turno=datos['TURNO']
        ciclo=datos['CICLO']
        if urltemp==url:
            writeForm(semi,semf,dia,doc,horae,horas,asig,turno,ciclo)
            print("")
        else:
            if str(semi)!="nan":   
                urltemp=url
                time.sleep(2)
                driver.get(url)
                time.sleep(1.5)
                writeForm(semi,semf,dia,doc,horae,horas,asig,turno,ciclo)
                print("")

            

1) Asig: AG63 - 583 - PROPAGACIÓN DE PLANTAS - 3253 VI M Docente :MATOS LIZANA JULIO CESAR SEM:6 semi:6 semf:16 FECHA:02052023
2) Asig: AG63 - 583 - PROPAGACIÓN DE PLANTAS - 3253 VI M Docente :MATOS LIZANA JULIO CESAR SEM:7 semi:6 semf:16 FECHA:09052023
3) Asig: AG63 - 583 - PROPAGACIÓN DE PLANTAS - 3253 VI M Docente :MATOS LIZANA JULIO CESAR SEM:8 semi:6 semf:16 FECHA:16052023
4) Asig: AG63 - 583 - PROPAGACIÓN DE PLANTAS - 3253 VI M Docente :MATOS LIZANA JULIO CESAR SEM:9 semi:6 semf:16 FECHA:23052023
5) Asig: AG63 - 583 - PROPAGACIÓN DE PLANTAS - 3253 VI M Docente :MATOS LIZANA JULIO CESAR SEM:10 semi:6 semf:16 FECHA:30052023
6) Asig: AG63 - 583 - PROPAGACIÓN DE PLANTAS - 3253 VI M Docente :MATOS LIZANA JULIO CESAR SEM:11 semi:6 semf:16 FECHA:06062023
7) Asig: AG63 - 583 - PROPAGACIÓN DE PLANTAS - 3253 VI M Docente :MATOS LIZANA JULIO CESAR SEM:12 semi:6 semf:16 FECHA:13062023
8) Asig: AG63 - 583 - PROPAGACIÓN DE PLANTAS - 3253 VI M Docente :MATOS LIZANA JULIO CESAR SEM:13 semi:6 sem

In [118]:
##SCRIPT PARA CARGA LECTIVA

import pandas as pd
df = pd.read_excel('cargalectiva.xlsx')
df
semanas_xpath= {'1':'//*[@id="exampleFormControlSelect1"]/option[1]',
               '2':'//*[@id="exampleFormControlSelect1"]/option[2]',
               '3':'//*[@id="exampleFormControlSelect1"]/option[3]',
               '4':'//*[@id="exampleFormControlSelect1"]/option[4]',
               '5':'//*[@id="exampleFormControlSelect1"]/option[5]',
               '6':'//*[@id="exampleFormControlSelect1"]/option[6]',
               '7':'//*[@id="exampleFormControlSelect1"]/option[7]',
               '8':'//*[@id="exampleFormControlSelect1"]/option[8]',
               '9':'//*[@id="exampleFormControlSelect1"]/option[9]',
               '10':'//*[@id="exampleFormControlSelect1"]/option[10]',
               '11':'//*[@id="exampleFormControlSelect1"]/option[11]',
               '12':'//*[@id="exampleFormControlSelect1"]/option[12]',
               '13':'//*[@id="exampleFormControlSelect1"]/option[13]',
               '14':'//*[@id="exampleFormControlSelect1"]/option[14]',
               '15':'//*[@id="exampleFormControlSelect1"]/option[15]',
               '16':'//*[@id="exampleFormControlSelect1"]/option[16]'}

try:
    for row, datos in df.iterrows():
        semana = datos['SEMANA']
        fecha = datos['FECHA']
        horae = datos['HORAE']
        horas = datos['HORAS']
        meet = datos['MEET']

        #ALGORITMO PARA LA FECHA
        f = str(fecha).split('-')
        año = f[0]
        mes = f[1]
        d= f[2].split()
        dia = d[0]
        fecha_resolve=[dia+mes+año]

        #ALGORTIMO HORA
        h1 = str(horae).split(':')
        horae=[h1[0]+h1[1]]
        h2 = str(horas).split(':')
        horas=[h2[0]+h2[1]]


        #ingresar semana
        driver.find_element(by=By.XPATH, value=semanas_xpath[str(int(semana))]).click()
        
        #ingresar fecha
        driver.find_element(by=By.XPATH, value='//*[@id="asistencia_docentes"]/div[4]/div[2]/table/tbody/tr[2]/td[2]/input').send_keys(fecha_resolve)
        
        #ingresar fecha entrada
        driver.find_element(by=By.XPATH, value='//*[@id="asistencia_docentes"]/div[4]/div[2]/table/tbody/tr[2]/td[3]/input').send_keys(str(horae))
        
        #ingresar fecha salida
        driver.find_element(by=By.XPATH, value='//*[@id="asistencia_docentes"]/div[4]/div[2]/table/tbody/tr[2]/td[4]/input').send_keys(str(horas))
        
        #ingresar meet
        driver.find_element(by=By.XPATH, value='//*[@id="asistencia_docentes"]/div[4]/div[2]/table/tbody/tr[2]/td[5]/input').clear()
        
        if  "nan" == str(meet) :
            print(str(""))
        else:
            driver.find_element(by=By.XPATH, value='//*[@id="asistencia_docentes"]/div[4]/div[2]/table/tbody/tr[2]/td[5]/input').send_keys(str(meet))
        
        
        #enviar registro
        driver.find_element(by=By.XPATH, value='//*[@id="asistencia_docentes"]/div[4]/div[2]/table/tbody/tr[2]/td[7]/button').click()
        time.sleep(0.200)
except:
    print("error")

In [4]:
##SCRIPT EXTRAER TODOS LOS LINK DE CARGA LECTIVA Y NO LECTIVA
import pandas as pd
from selenium.webdriver.common.action_chains import ActionChains
import re
from selenium.common.exceptions import NoSuchElementException
import time
asig = []
ciclo = []
t = []
s = []
c = []
docen = []
cl = []
cnl = []
av=[]
baseURL="https://sivireno.undc.edu.pe/"
##la cantidad que te muestra el sivireno
cantlist=100

###########################################CREACION DE METODOS#################################################################

def addLinkExcel():
    df = pd.DataFrame({ 'ASIGNATURA': asig,'CICLO': ciclo,'TURNO': t,'SECCION': s,'CANT': c,'DOCENTE': docen,'CL': cl,'CNL': cnl,'AV':av})
    df = df[['ASIGNATURA', 'CICLO','TURNO','SECCION','CANT','DOCENTE','CL','CNL','AV']]
    writer = pd.ExcelWriter('EXCEL_GENERADOS/CREATE_LIST_CURSOS_AGRONOMIA'+time.strftime("%Y%m%d-%H-%M-%S")+'.xlsx')
    df.to_excel(writer, 'Sheet', index=False)
    #writer.save()
    writer.close()
    print("TERMINO PROCESO")
###########################################CREACION DE METODOS#################################################################
try:
    for j in range(int(cantlist)):
        
        #SOLO PARA LA MODALIDAD DEL CURSO EN M
                                                    
        if driver.find_element(by=By.XPATH, value='//*[@id="app"]/div[2]/table/tbody/tr['+str(j+1)+']/td[3]').text =="M":
            asig.append(driver.find_element(by=By.XPATH, value='//*[@id="app"]/div[2]/table/tbody/tr['+str(j+1)+']/td[4]').text)
            ciclo.append(driver.find_element(by=By.XPATH, value='//*[@id="app"]/div[2]/table/tbody/tr['+str(j+1)+']/td[5]').text)
            t.append(driver.find_element(by=By.XPATH, value='//*[@id="app"]/div[2]/table/tbody/tr['+str(j+1)+']/td[6]').text)
            s.append(driver.find_element(by=By.XPATH, value='//*[@id="app"]/div[2]/table/tbody/tr['+str(j+1)+']/td[7]').text)
            c.append(driver.find_element(by=By.XPATH, value='//*[@id="app"]/div[2]/table/tbody/tr['+str(j+1)+']/td[8]').text)
            docen.append(driver.find_element(by=By.XPATH, value='//*[@id="app"]/div[2]/table/tbody/tr['+str(j+1)+']/td[9]').text)
            cl.append(baseURL+driver.execute_script('return document.querySelector("#app > div.table-responsive > table > tbody > tr:nth-child('+str(j+1)+') > td:nth-child(10) > a:nth-child(1)").getAttribute("href");'))
            cnl.append(baseURL+driver.execute_script('return document.querySelector("#app > div.table-responsive > table > tbody > tr:nth-child('+str(j+1)+') > td:nth-child(10) > a:nth-child(2)").getAttribute("href");'))
            av.append(driver.execute_script('return document.querySelector("#app > div.table-responsive > table > tbody > tr:nth-child('+str(j+1)+') > td:nth-child(13) > a").getAttribute("href");'))
                                                        
    addLinkExcel()
except:
    addLinkExcel()


TERMINO PROCESO


In [76]:
import os
simp_path = '.env'
abs_path = os.path.abspath(simp_path)
CREDENTIAL={'EMAIL':'','PASSWORD':''}
with open(abs_path,"r") as archivo:
    for linea in archivo:        
        if (str(linea.strip('\n')).split('='))[0] == "EMAIL":
            CREDENTIAL["EMAIL"]=(str(linea.strip('\n')).split('='))[1]
        if (str(linea.strip('\n')).split('='))[0] == "PASSWORD":
            CREDENTIAL["PASSWORD"]=(str(linea.strip('\n')).split('='))[1]


soporteagronomia@undc.edu.pe
